<a href="https://colab.research.google.com/github/deep-diver/deeplearning-with-structured-data/blob/master/notebooks/streetcar_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/deep-diver/deeplearning-with-structured-data.git
!mv deeplearning-with-structured-data/* ./

Cloning into 'deeplearning-with-structured-data'...
remote: Enumerating objects: 277, done.
remote: Counting objects: 100% (277/277), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 277 (delta 115), reused 263 (delta 106), pack-reused 0
Receiving objects: 100% (277/277), 21.36 MiB | 25.03 MiB/s, done.
Resolving deltas: 100% (115/115), done.


# 경전철 지연 예측 - 데이터 준비

미래의 지연을 예측을 예측하고, 지연 발생에 대한 조언을 구하고자 토론토 교통국(TTC)의 경전철 지연 데이터 중 2014년 - 현재까지를 포함한 데이터셋을 사용합니다.

원본 데이터셋의 위치: https://open.toronto.ca/dataset/ttc-streetcar-delay-data/

이 노트북은 일반적인 데이터 불러오기, 준비 과정을 다룹니다:
- 모든 XLS 파일의 모든 스프레드시트를 단일 데이터프레임으로 만듭니다
- 데이터 유형의 문제를 수정
- 누락된 값 고치기
- 경로(route), 위치(location), 방향(direction), 차량(vehicle) 열의 잘못된 값을 정리합니다

# 경전철 경로(routes)

출처: https://www.ttc.ca/Routes/Streetcars.jsp

<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/streetcarnov3/master/streetcar%20routes.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

# 공통 라이브러리 및 변수
노트북 전체에서 공통적으로 사용되는 변수를 정의하고 라이브러리를 불러옵니다

In [ ]:
!pip install requests
!pip install xlrd

In [ ]:
# 선형 대수용 라이브러리
import numpy as np 

# 데이터 처리용 라이브러리로 CSV 파일 입출력을 위해 불러옵니다(예. pd.read_csv)
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

# 공통 라이브러리
import zipfile
import time
# import datetime, timedelta
import datetime
from datetime import datetime, timedelta
from datetime import date
from dateutil import relativedelta
from io import StringIO
import pandas as pd
import pickle
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from io import StringIO
import requests
import json
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline 
import os
import math
from subprocess import check_output
from IPython.display import display
import logging
import yaml
from collections import Counter
import re
import os


In [ ]:
# 설정 파일을 불러옵니다
current_path = os.getcwd()
# Google Colab 외 환경에서는 아래의 코드를 주석처리 합니다
current_path = os.path.join(current_path, "notebooks")
print("현재 디렉터리: "+current_path)

path_to_yaml = os.path.join(current_path, 'streetcar_data_preparation_config.yml')
print("yaml 파일 경로(path_to_yaml): "+path_to_yaml)
try:
    with open (path_to_yaml, 'r') as c_file:
        config = yaml.safe_load(c_file)
except Exception as e:
    print('설정 파일을 읽는 데 오류가 발생했습니다')
    

현재 디렉터리: /content/notebooks
yaml 파일 경로(path_to_yaml): /content/notebooks/streetcar_data_preparation_config.yml


In [ ]:

# 공통 변수
# 원본 또는 저장된 데이터프레임을 불러올지를 제어합니다
load_from_scratch = config['general']['load_from_scratch']

# 변형된 데이터로 데이터프레임을 저장할지를 제어합니다
save_transformed_dataframe = config['general']['save_transformed_dataframe']

# 저장된 데이터셋에서 오류가 있는 값을가진 행을 제거할지를 제어합니다
remove_bad_values = config['general']['remove_bad_values']

# 피클로 저장된 입력 데이터셋의 파일명 (전처리 전)
pickled_input_dataframe = config['file_names']['pickled_input_dataframe']

# 피클로 저장된 입력 데이터셋의 파일명 (전처리 후)
pickled_output_dataframe = config['file_names']['pickled_output_dataframe']

In [ ]:
print("load_from_scratch 제어 변수: "+str(load_from_scratch))
print("save_transformed_dataframe 제어 변수: "+str(save_transformed_dataframe))
print("remove_bad_values 제어 변수: "+str(remove_bad_values))
print("pickled_input_dataframe 파일명: "+str(pickled_input_dataframe))
print("pickled_output_dataframe 파일명: "+str(pickled_output_dataframe))

load_from_scratch 제어 변수: False
save_transformed_dataframe 제어 변수: True
remove_bad_values 제어 변수: True
pickled_input_dataframe 파일명: 2014_2019.pkl
pickled_output_dataframe 파일명: 2014_2019_df_cleaned_remove_bad_values_may16_2020.pkl


# CLRV/ALRV용 경전철 차량 식별자(IDs)

출처: https://en.wikipedia.org/wiki/Toronto_streetcar_system_rolling_stock#CLRVs_and_ALRVs

<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/streetcarnov3/master/streetcarCLRV.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

In [ ]:
# CLRV/ALRV에 맞는 차량 식별자들을 모읍니다
streetcar_vehicles = list(range(4000,4006))+ list(range(4010,4200)) +  list(range(4200,4252)) + [4900]
streetcar_vehicles = streetcar_vehicles + [4400] + list(range(4402,4508))

print("CLRV/ALRV용 유효한 경전철 목록",streetcar_vehicles)

CLRV/ALRV용 유효한 경전철 목록 [4000, 4001, 4002, 4003, 4004, 4005, 4010, 4011, 4012, 4013, 4014, 4015, 4016, 4017, 4018, 4019, 4020, 4021, 4022, 4023, 4024, 4025, 4026, 4027, 4028, 4029, 4030, 4031, 4032, 4033, 4034, 4035, 4036, 4037, 4038, 4039, 4040, 4041, 4042, 4043, 4044, 4045, 4046, 4047, 4048, 4049, 4050, 4051, 4052, 4053, 4054, 4055, 4056, 4057, 4058, 4059, 4060, 4061, 4062, 4063, 4064, 4065, 4066, 4067, 4068, 4069, 4070, 4071, 4072, 4073, 4074, 4075, 4076, 4077, 4078, 4079, 4080, 4081, 4082, 4083, 4084, 4085, 4086, 4087, 4088, 4089, 4090, 4091, 4092, 4093, 4094, 4095, 4096, 4097, 4098, 4099, 4100, 4101, 4102, 4103, 4104, 4105, 4106, 4107, 4108, 4109, 4110, 4111, 4112, 4113, 4114, 4115, 4116, 4117, 4118, 4119, 4120, 4121, 4122, 4123, 4124, 4125, 4126, 4127, 4128, 4129, 4130, 4131, 4132, 4133, 4134, 4135, 4136, 4137, 4138, 4139, 4140, 4141, 4142, 4143, 4144, 4145, 4146, 4147, 4148, 4149, 4150, 4151, 4152, 4153, 4154, 4155, 4156, 4157, 4158, 4159, 4160, 4161, 4162, 4163, 4164, 4165, 4166,

# 경전철 차량 식별자(IDs) 범위의 특이사항

출처: https://en.wikipedia.org/wiki/Toronto_streetcar_system_rolling_stock#CLRVs_and_ALRVs

<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/streetcarnov3/master/streetcarflexity.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

# 버스 식별자

아래의 링크는 경전철에 의해 지연될 수 있는, 경전철이 아닌 유효 차량을 정의합니다

- 버스 1xxx: https://cptdb.ca/wiki/index.php/Toronto_Transit_Commission_1000-1149
- 버스 2xxx: https://cptdb.ca/wiki/index.php/Toronto_Transit_Commission_2000-2110,_2150-2155,_2240-2485,_2600-2619,_2700-2765,_2767-2858
- 버스 70xx: https://cptdb.ca/wiki/index.php/Toronto_Transit_Commission_7000-7134
- 버스 74xx: https://cptdb.ca/wiki/index.php/Toronto_Transit_Commission_7400-7499,_7500-7619,_7620-7881
- 버스 8xxx: https://cptdb.ca/wiki/index.php/Toronto_Transit_Commission_8000-8099
- 버스 9xxx: https://cptdb.ca/wiki/index.php/Toronto_Transit_Commission_9000-9026







In [ ]:
bus_vehicles = list(range(1000,1150))+ list(range(2000,2111)) + list(range(2150,2156)) + list(range(2240,2486))
bus_vehicles = bus_vehicles + list(range(2600,2620)) + list(range(2700,2766)) + list(range(2767,2859))
bus_vehicles = bus_vehicles + list(range(7000,7135)) + list(range(7400,7450)) + list(range(7500,7620)) + list(range(7620,7882))
bus_vehicles = bus_vehicles + list(range(8000,8100)) + list(range(9000,9027))
valid_vehicles = streetcar_vehicles + bus_vehicles

print("유효한 차량 목록",valid_vehicles)

유효한 차량 목록 [4000, 4001, 4002, 4003, 4004, 4005, 4010, 4011, 4012, 4013, 4014, 4015, 4016, 4017, 4018, 4019, 4020, 4021, 4022, 4023, 4024, 4025, 4026, 4027, 4028, 4029, 4030, 4031, 4032, 4033, 4034, 4035, 4036, 4037, 4038, 4039, 4040, 4041, 4042, 4043, 4044, 4045, 4046, 4047, 4048, 4049, 4050, 4051, 4052, 4053, 4054, 4055, 4056, 4057, 4058, 4059, 4060, 4061, 4062, 4063, 4064, 4065, 4066, 4067, 4068, 4069, 4070, 4071, 4072, 4073, 4074, 4075, 4076, 4077, 4078, 4079, 4080, 4081, 4082, 4083, 4084, 4085, 4086, 4087, 4088, 4089, 4090, 4091, 4092, 4093, 4094, 4095, 4096, 4097, 4098, 4099, 4100, 4101, 4102, 4103, 4104, 4105, 4106, 4107, 4108, 4109, 4110, 4111, 4112, 4113, 4114, 4115, 4116, 4117, 4118, 4119, 4120, 4121, 4122, 4123, 4124, 4125, 4126, 4127, 4128, 4129, 4130, 4131, 4132, 4133, 4134, 4135, 4136, 4137, 4138, 4139, 4140, 4141, 4142, 4143, 4144, 4145, 4146, 4147, 4148, 4149, 4150, 4151, 4152, 4153, 4154, 4155, 4156, 4157, 4158, 4159, 4160, 4161, 4162, 4163, 4164, 4165, 4166, 4167, 4168,

# 경전철 사건과는 무관한 차량
아래의 차량은 완전히 분리된 도로를 사용하거나, 이제는 퇴역(사용되지않는) 버스로 경전철 사건과는 무관합니다.

- RT 차량 3xxx: https://cptdb.ca/wiki/index.php/Toronto_Transit_Commission_3000-3027
- 지하철 차량 5xxx https://cptdb.ca/wiki/index.php/Toronto_Transit_Commission_5000-5371
- 퇴역 차량 6xxx: https://cptdb.ca/wiki/index.php/Toronto_Transit_Commission_6000-6122


In [ ]:
# 유효한 TTC 경전철 경로 목록
valid_routes = ['501','502','503','504','505','506','509','510','511','512','301','304','306','310']

In [ ]:
valid_routes

['501',
 '502',
 '503',
 '504',
 '505',
 '506',
 '509',
 '510',
 '511',
 '512',
 '301',
 '304',
 '306',
 '310']

In [ ]:
# 유효한 방향 (원본)
# valid_directions = ['E/B','W/B','N/B','S/B','B/W']
# 여기서 '/' 문자를 제거한 뒤 소문자 처리를 합니다 (간소화 작업)
valid_directions = ['e','w','n','s','b']

In [ ]:
valid_days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

# 데이터 불러오기와 저장하기
- XLS 파일 목록을 파싱합니다
- XLS 파일에 포함된 스프레드를 데이터프레임으로 만듭니다
- 이후 작업을 위해 데이터프레임을 피클로 저장합니다

In [ ]:
# 현재 노트북이 담긴 디렉터리를 가져온 뒤, 이에 기반하여 데이터가 포함된 디렉터리를 반환합니다

def get_path():
    rawpath = os.getcwd()
    # 데이터는 "data" 디렉터리에 들어있으며, "notebooks" 디렉터리와 같은 계위에 위치합니다
    # 지금부터의 코드는 모든 XLS 파일이 "data" 디렉터리에 존재한다고 가정합니다.
    # 예제소스가 담긴 저장소를 복제하면 모든 파일이 이미 들어있습니다.
    path_to_yaml = os.path.join(current_path, 'notebooks', 'streetcar_data_preparation_config.yml')

    # Google Colab을 사용하지 않는 경우, 아래의 코드를 다음과 같이 대체합니다
    # path = os.path.abspath(os.path.join(rawpath, '..', 'data'))
    path = os.path.abspath(os.path.join(rawpath, 'data'))
    return(path)


In [ ]:
# 주어진 경로에 기반해 모든 xls 파일 목록을 반환합니다
def get_xls_list(path):
    files = os.listdir(path)
    files_xls = [f for f in files if f[-4:] == 'xlsx']
    print(files)
    print(files_xls)
    return(files_xls)


In [ ]:
def load_xls(path, files_xls, firstfile, firstsheet, df):
    '''
    모든 XLS 파일의 모든 탭(스프레드시트)를 불러옵니다.
    단 최초 데이터프레임을 생성할 때 사용한 스프레드시트는 이 과정에서 제외합니다
    
    파라미터:
    path: XLS 파일을 담은 경로
    files_xls: XLS 파일 목록(리스트)
    firstfile: 이 함수가 호출되기 전 이미 불러와진, 첫 번째 탭을 소유한 파일명
    firstsheet: 이 함수가 호출되기 전 이미 불러와진 첫 번째 탭의 이름
    df: 이 함수가 호출되기 전 이미 불러와진 첫 번째 탭을 포함하고 있다가,
        이 함수가 종료될 때는 모든 데이터를 이어붙여 완성된 팬더스 데이터프레임
    
    반환:
    df: 갱신된 데이터프레임
    
    '''
    for f in files_xls:
        print("file name",f)
        xlsf = pd.ExcelFile(os.path.join(path,f))
        # 포함된 모든 스프레드시트를 반복적으로 접근합니다
        for sheet_name in xlsf.sheet_names:
            print("스프레드시트 이름(sheet_name):",sheet_name)
            if (f != firstfile) or (sheet_name != firstsheet):
                print("대상 스프레드시트 이름:",sheet_name)
                data = pd.read_excel(os.path.join(path,f),sheet_name=sheet_name)    
                df = df.append(data)
    return (df)

In [ ]:
# 주어진 경로내 모든 XLS 파일을 불러온 후
# 주어진 파일명으로 데이터프레임을 저장합니다
def reloader(path,picklename):
    # 경로내 모든 XLS 파일 목록을 가져옵니다
    files_xls = get_xls_list(path)
    print("xls 파일 목록",files_xls)

    # 첫 번째 XLS 파일내 첫 번째 탭(스프레드시트)로 초기 데이터프레임을 만듭니다
    dfnew = pd.read_excel(os.path.join(path,files_xls[0]))

    # 첫 번째 파일내 스프레드시트 목록을 가져옵니다
    xlsf = pd.ExcelFile(os.path.join(path,files_xls[0]))

    # 다른 모든 XLS 파일내 모든 탭을 불러옵니다
    # 이 때 앞서 정의한 load_xls 함수에는 첫 번째 XLS 파일과, 초기 데이터프레임 구성에 사용된 탭 이름을 전달합니다
    dflatest = load_xls(path,files_xls,files_xls[0],xlsf.sheet_names[0], dfnew)

    # 구축된 데이터프레임을 피클로 저장합니다
    dflatest.to_pickle(os.path.join(path,picklename))

    # 모든 XLS 파일의 모든 탭을 담은 데이터프레임을 반환합니다
    return(dflatest)
    

In [ ]:
# 입력 열들을 유형에 맞게 분류합니다
def define_feature_categories(df):
    allcols = list(df)
    print("전체 열 목록:",allcols)
    textcols = ['Incident','Location'] 
    continuouscols = ['Min Delay','Min Gap'] 
                      # 연속형 값을 다루는 열 - 이후(모델링시) 이 열에는 임베딩을 사용하지 않습니다
    timecols = ['Report Date','Time']
    collist = ['Day','Vehicle','Route','Direction']
    for col in continuouscols:
        df[col] = df[col].astype(float)
    print('텍스트 열 목록(textcols): ',textcols)
    print('연속형 열 목록(continuouscols): ',continuouscols)
    print('시간형 열 목록(timecols): ',timecols)
    print('범주형 열 목록(collist): ',collist)
    return(allcols,textcols,continuouscols,timecols,collist)

In [ ]:
# 열 유형에 맞게 누락된 값을 채워넣습니다
def fill_missing(dataset,allcols,textcols,continuouscols,timecols,collist):
    logging.debug("before mv")
    for col in collist:
        dataset[col].fillna(value="missing", inplace=True)
    for col in continuouscols:
        dataset[col].fillna(value=0.0,inplace=True)
    for col in textcols:
        dataset[col].fillna(value="missing", inplace=True)
    return (dataset)

# 데이터프레임 불러오기
- 피클로 저장된 데이터프레임을 불러옵니다
- 해당 데이터셋의 정보를 출력합니다

In [ ]:
# 원본 XLS 파일 또는 피클로 저장된 데이터프레임의 데이터를 읽습니다
def ingest_data(path):
    if load_from_scratch:
        unpickled_df = reloader(path,pickled_input_dataframe)
        logging.debug("재읽기 완료")
    else:
        unpickled_df = pd.read_pickle(os.path.join(path,pickled_input_dataframe))
    return(unpickled_df)

# 일반적인 정리 작업
- 경로(Route) 및 차량(Vehicle)의 잘못된 유형 바로잡기
- 누락된 값 채워넣기
- report-date-time 인덱스 생성하기

In [ ]:
# 2019년도의 데이터에는 일부 이상한 값들이 포함되어 있습니다. 가령,
# 2019년도 4월 탭(스프레드시트)에는 무의미한 Incident ID 열이 들어있고,
# 2019년도 4, 6월 탭에는 Min Gap, Min Delay로 표시되어야 할 열이 Gap, Delay로 되어있던 문제가 있습니다.
# 아래의 함수는 이런 문제를 정리합니다
def fix_anomalous_columns(df):
    # Min Delay나 Min Gap 열에 NaN 값이 있는 행이 있다면, Delay나 Gap 으로부터 값을 복사해서 채워넣습니다.
    # df.Temp_Rating.fillna(df.Farheit, inplace=True)
    df['Min Delay'].fillna(df['Delay'], inplace=True)
    df['Min Gap'].fillna(df['Gap'], inplace=True)
    # now that the useful values have been copied from Delay and Gap, remove them
    del df['Delay']
    del df['Gap']
    # 무의미한 Incident ID 열을 삭제합니다
    del df['Incident ID']
    return(df)

In [ ]:
def replace_time(date_time_value,time_value):
    ''' 주어진 datetime으로 시간 부분을 대체합니다 '''
     
    date_time_value = date_time_value.replace(hour=time_value.hour,minute=time_value.minute,second=time_value.minute)
    return(date_time_value)


In [ ]:
def general_cleanup(df):
    # Route와 Vehicle열 값이 문자열이 되도록 강제합니다
    df['Route'] = df['Route'].astype(str)
    df['Vehicle'] = df['Vehicle'].astype(str)
    # 부동소수를 문자열로 바꾸며 생긴 무의미한 문자(".0")를 제거합니다
    df['Vehicle'] = df['Vehicle'].str[:-2]
    # 열 유형을 분류해 각자의 변수로 저장합니다
    allcols,textcols,continuouscols,timecols,collist = define_feature_categories(df)
    # 누락된 값을 채워넣습니다
    df.isnull().sum(axis = 0)
    df = fix_anomalous_columns(df)
    df = fill_missing(df,allcols,textcols,continuouscols,timecols,collist)
    # 날짜 + 시간을 조함한 새로운 열을 생성한 뒤, 이를 인덱스로 사용합니다
    df['Report Date Time'] = df.apply(lambda x: replace_time(x['Report Date'], x['Time']), axis=1)
    df.index = df['Report Date Time']
    # 갱신된 데이터프레임과 열 유형별 목록을 반환합니다
    return(df,allcols,textcols,continuouscols,timecols,collist)

# 선택된 열 정리 작업

입력 데이터셋의 일부 값은 "자유 형식"으로 입력되었습니다. 이런 값을 가진 열에는 다음과 같은것이 있습니다.

- Route
- Vehicle
- Direction
- Location

각 열은 유한한 고윳값 목록을 가집니다(범주형). 이 열들에 담긴 데이터 중 같은것을 의미하지만 서로다른 토큰이 할당된 문제를 해결해야 합니다(예. "roncesvalles yard"와 "roncesvalles carhouse"). 또한 잘못된 (유효하지 않은) 값이 입력된 문제도 해결되야 합니다(예. 나침반에 없는 방향 값을 가진 Direction). 

# Clean up Route

In [ ]:
def check_route (x):
    if x in valid_routes:
        return(x)
    else:
        return("bad route")

In [ ]:
def route_cleanup(df):
    print("정리 전 Route의 고윳값 개수: ",df['Route'].nunique())
    # df['Route'].value_counts()
    # 잘못된 경로(route)를 공통 토큰(bad route)으로 대체합니다
    df['Route'] = df['Route'].apply(lambda x:check_route(x))
    print("정리 후 Route의 고윳값 개수: ",df['Route'].nunique())
    return(df)    

# Vehicle(차량 식별자) 열 정리

In [ ]:
def check_vehicle (x):
    if str.isdigit(x):
        if int(x) in valid_vehicles:
            return x
        else:
            return("bad vehicle")
    else:
        return("bad vehicle")

In [ ]:
def vehicle_cleanup(df):
    print("정리 전 Vehicle의 고윳값 개수: ",df['Vehicle'].nunique())
    df['Vehicle'] = df['Vehicle'].apply(lambda x:check_vehicle(x))
    print("정리 후 Vehicle의 고윳값 개수",df['Vehicle'].nunique())
    return(df)

# Direction(방향) 열 정리

In [ ]:
def check_direction (x):
    if x in valid_directions:
        return(x)
    else:
        return("bad direction")

In [ ]:
def direction_cleanup(df):
    print("정리 전 Direction의 고윳값 개수: ",df['Direction'].nunique())
    df['Direction'] = df['Direction'].str.lower()
    df['Direction'] = df['Direction'].str.replace('/','')
    df['Direction'] = df['Direction'].replace({'eastbound':'e','westbound':'w','southbound':'s','northbound':'n'})
    df['Direction'] = df['Direction'].replace('b','',regex=True)
    df['Direction'] = df['Direction'].apply(lambda x:check_direction(x))
    print("정리 후 Direction의 고윳값 개수: ",df['Direction'].nunique())
    return(df)

# Location(위치) 열 정리

In [ ]:
def clean_conjunction(intersection):
    intersection = re.sub(" *& *"," and ",intersection)
    intersection = re.sub(" */ *"," and ",intersection)
    return(intersection)

In [ ]:
def order_location(intersection):
    # " and "가 중간에 들어있는 모든 문자열에 대해
    # 만약 " and " 앞에 등장한 문자열 값이 뒤에 등장한 문자열보다 
    # 알파벳 순서가 더 높다면, 이 둘의 위치를 뒤바꿉니다
    conj = " and "
    alpha_ordered_intersection = intersection
    if conj in intersection:
        end_first_street = intersection.find(conj)
        if (end_first_street > 0) and (len(intersection) > (end_first_street + len(conj))):
            start_second_street = intersection.find(conj) + len(conj)
            first_street = intersection[0:end_first_street]
            second_street = intersection[start_second_street:]
            alpha_ordered_intersection = min(first_street,second_street)+conj+max(first_street,second_street)
    return(alpha_ordered_intersection)

In [ ]:
def location_cleanup(df):
    print("정리 전 Location의 고윳값 개수: ",df['Location'].nunique())
    # Location 의 모든 값을 소문자로 만듭니다
    df['Location'] = df['Location'].str.lower()
    # 중복 토큰을 하나의 토큰으로 대체합니다
    df['Location'] = df['Location'].replace({'broadviewstation':'broadview station',' at ':' and ',' stn':' station',' ave.':'','/':' and ','roncy':'roncesvalles','carhouse':'yard','yard.':'yard','st. clair':'st clair','ronc. ':'roncesvalles ','long branch':'longbranch','garage':'yard','barns':'yard',' & ':' and '}, regex=True)
    # 교차로 문자열 값의 알파벳 순서를 바로잡습니다
    df['Location'] = df['Location'].apply(lambda x:order_location(x))
    print("정리 후 Location의 고윳값 개수: ",df['Location'].nunique())
    return(df)

# 잘못된 값을 가진 행 삭제

In [ ]:
# 잘못된 값을 가진 행을 삭제합니다
def remove_bad(df):
    df = df[df.Vehicle != 'bad vehicle']
    df = df[df.Direction != 'bad direction']
    df = df[df.Route != 'bad route']
    return(df)

In [ ]:
'''
# get the path for data files
path = get_path()
pickled_input_dataframe = '2014_2019_upto_june_from_repo.pkl'
print("path is ",path)
# load route direction and delay data datframes
df = ingest_data(path)
df.head()
'''

'\n# get the path for data files\npath = get_path()\npickled_input_dataframe = \'2014_2019_upto_june_from_repo.pkl\'\nprint("path is ",path)\n# load route direction and delay data datframes\ndf = ingest_data(path)\ndf.head()\n'

# Master cell

앞서 정의한 모든 정리용 함수를 호출해서, 모든 데이터 정리 작업을 수행합니다.

In [ ]:
# 모든 함수를 호출하는 마스터 셀

# 데이터 파일이 들어있는 경로를 가져옵니다
path = get_path()
print("경로는 ",path)

# route, direction, delay 데이터를 데이터프레임 형식으로 불러옵니다
df = ingest_data(path)
print("전체 데이터 수: ",len(df.index))
print("df.info() 출력 내용",df.info())
print("df.shape 출력 내용",df.shape)
print("df.describe() 출력 내용",df.describe())
print("df.types 출력 내용",df.dtypes)
df,allcols,textcols,continuouscols,timecols,collist = general_cleanup(df)
df.head()

# 연도별 데이터 수를 구합니다
from collections import Counter
df_year = pd.DatetimeIndex(df['Report Date Time']).year
print("데이터 전처리 전, 연도별 데이터 개수: ", str(Counter(df_year)))

# 2019년도 4월의 값이 올바른지 검사합니다
df[df['Report Date Time'].astype(str).str[:7]=='2019-04']

# 정리 작업
logging.debug("정리 전 df.shape 출력 내용",df.shape)
df = route_cleanup(df) 
df = vehicle_cleanup(df)
df = direction_cleanup(df)
df = location_cleanup(df)
logging.debug("정리 후 df.shape 출력 내용",df.shape)
print("정리 전 'bad route'의 개수:",df[df.Route == 'bad route'].shape[0])
print("정리 전 'bad direction'의 개수:",df[df.Direction == 'bad direction'].shape[0])
print("정리 전 'bad vehicle'의 개수:",df[df.Vehicle == 'bad vehicle'].shape[0])
if remove_bad_values:
    df = remove_bad(df)
print("정리 후 'bad route'의 개수:",df[df.Route == 'bad route'].shape[0])
print("정리 후 'bad direction'의 개수:",df[df.Direction == 'bad direction'].shape[0])
print("정리 후 'bad vehicle'의 개수:",df[df.Vehicle == 'bad vehicle'].shape[0])

# 정리된 데이터프레임을 피클로 저장합니다
print("잘못된 기록을 삭제한 후 df.shape 출력 내용 ",df.shape)
if save_transformed_dataframe:
    print("경로는 ",path)
    file_name = os.path.join(path,pickled_output_dataframe)
    print("파일명(file_name)은 ",file_name)
    df.to_pickle(file_name)
df.head()

path is  C:\personal\manning\deep_learning_for_structured_data\data
number of records:  78525
<class 'pandas.core.frame.DataFrame'>
Int64Index: 78525 entries, 0 to 814
Data columns (total 13 columns):
Day            78525 non-null object
Delay          3444 non-null float64
Direction      78217 non-null object
Gap            3434 non-null float64
Incident       78525 non-null object
Incident ID    889 non-null float64
Location       78276 non-null object
Min Delay      75004 non-null float64
Min Gap        74975 non-null float64
Report Date    78525 non-null datetime64[ns]
Route          78525 non-null int64
Time           78525 non-null object
Vehicle        73890 non-null float64
dtypes: datetime64[ns](1), float64(6), int64(1), object(5)
memory usage: 8.4+ MB
df.info() output None
df.shape output (78525, 13)
df.describe() output              Delay          Gap  Incident ID     Min Delay       Min Gap  \
count  3444.000000  3434.000000   889.000000  75004.000000  74975.000000   
mean 

,Day,Direction,Incident,Location,Min Delay,Min Gap,Report Date,Route,Time,Vehicle,Report Date Time
Report Date Time,,,,,,,,,,,
2014-01-02 06:31:31,Thursday,e,Late Leaving Garage,dundas and roncesvalles,4.0,8.0,2014-01-02,505,06:31:00,4018,2014-01-02 06:31:31
2014-01-02 12:43:43,Thursday,e,Utilized Off Route,king and shaw,20.0,22.0,2014-01-02,504,12:43:00,4128,2014-01-02 12:43:43
2014-01-02 14:01:01,Thursday,w,Held By,bingham and kingston road,13.0,19.0,2014-01-02,501,14:01:00,4016,2014-01-02 14:01:01
2014-01-02 14:22:22,Thursday,w,Investigation,king st. and roncesvalles,7.0,11.0,2014-01-02,504,14:22:00,4175,2014-01-02 14:22:22
2014-01-02 16:42:42,Thursday,e,Utilized Off Route,bathurst and king,3.0,6.0,2014-01-02,504,16:42:00,4080,2014-01-02 16:42:42


In [ ]:
# get record count by year
from collections import Counter
df_year = pd.DatetimeIndex(df['Report Date Time']).year
Counter(df_year)

Counter({2014: 9371,
         2015: 10898,
         2016: 11908,
         2017: 9891,
         2018: 12011,
         2019: 7474})